# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

 20%|██        | 1/5 [01:25<05:42, 85.56s/it]c:\Users\USER\Documents\GitHub\llm_practice\week8\agents\deals.py:27: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  description = BeautifulSoup(description, 'html.parser').get_text()
100%|██████████| 5/5 [08:16<00:00, 99.22s/it] 


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

'Title: Taimasi 25-Piece Organizer Set for $16 + free shipping w/ $35\nDetails: Use it to organize makeup, jewelry, and gadgets. It\'s $24 off. Shipping adds $6.99 or orders of $35 or more ships for free. Buy Now at Walmart\nFeatures: three 9" x 6" x 2.2" bins six 9" x 3" x 2.2" bins eight 6" x 3" x 2.2" bins eight 3" x 3" x 2.2"\nURL: https://www.dealnews.com/Taimasi-25-Piece-Organizer-Set-for-16-free-shipping-w-35/21718744.html?iref=rss-c196'

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Samsung DU6900 65" Crystal UHD 4K HDR Smart TV for $350 + free shipping
Details: That's $28 under our December mention and the lowest price we've ever seen. Buy Now at Best Buy
Features: 4K resolution  HDR  Samsung Tizen OS Model: UN65DU6900FXZA
URL: https://www.dealnews.com/products/Samsung/Samsung-DU6900-65-Crystal-UHD-4-K-HDR-Smart-TV/484484.html?iref=rss-c142

T

In [9]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [10]:
result = get_recommendations()

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [11]:
len(result.deals)

NameError: name 'result' is not defined

In [12]:
result.deals[1]

NameError: name 'result' is not defined

In [13]:
from agents.scanner_agent import ScannerAgent

In [14]:
agent = ScannerAgent()
result = agent.scan()

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
result